In [10]:
import generative_model
import numpy as np

In [2]:
params = {
    "NOISE": 0.1,
    "NO_OF_TRAJECTORIES": 1000,
    "CONST_VEL_MODEL_PROB": 0.8,
    
    "STOP_PROB": 0.2,
    
    "DISCOUNT_AVG_PROB": 0.5,
    "DISCOUNT_LOWER_BOUND": 0.5,
    
    "VELOCITY_CHANGE_PROB": 0.2,
    "VELOCITY_CHANGE_NOISE": 0.1,
    
    "ANGLE_CHANGE_PROB": 0.05,
    "ANGLE_CHANGE_NOISE": 1,
    
    "CLUSTER_PERCENTAGES": [0.2, 0.68, 0.95, 1.0]
}

In [3]:
import json
import pandas as pd

In [39]:
import trajnetplusplustools
from trajnetplusplustools.data import TrackRow
test_path = 'trajnet_data/test_original/'
pred_path = 'trajnet_data/test/'
file_paths = [
    'real_data/biwi_eth',
    'real_data/crowds_uni_examples',
    'real_data/crowds_zara02',
    'synth_data/collision_test',
    'synth_data/orca_synth'
]
sample = 1.0

In [41]:
for file_path in file_paths:

    reader = trajnetplusplustools.Reader(test_path + file_path + '.ndjson', scene_type='paths')
    scenes = [(s_id, s) for s_id, s in reader.scenes(sample=sample)]

    for scene_data in scenes:
        scene_id = scene_data[0]
        pedestrian_tracks = scene_data[1]

        primary_pedestrian_track = pedestrian_tracks[0]
        pedestrian_id = primary_pedestrian_track[0].pedestrian
        last_frame_no = primary_pedestrian_track[-1].frame

        xs = []
        ys = []
        for frame in primary_pedestrian_track:
            frame_id = frame.frame
            x = frame.x
            y = frame.y

            xs.append(x)
            ys.append(y)

        all_pred_x, all_pred_y, weights = generative_model.predict(xs, ys, params, trajectory_length=12)
        sorting_order = np.array(weights).argsort()[::-1]
        all_pred_x = np.array(all_pred_x)[sorting_order]
        all_pred_y = np.array(all_pred_y)[sorting_order]

        for prediction_no in range(len(all_pred_x)):
            pred_x = all_pred_x[prediction_no]
            pred_y = all_pred_y[prediction_no]

            for i in range(len(pred_x)):
                pred_row = TrackRow(
                    frame=last_frame_no + i + 1,
                    pedestrian=pedestrian_id,
                    x=pred_x[i],
                    y=pred_y[i],
                    prediction_number=prediction_no,
                    scene_id=scene_id
                )

                pred_row = trajnetplusplustools.writers.trajnet_tracks(pred_row)
                with open(pred_path + file_path + '.ndjson', "a") as pred_file:
                    pred_file.write(pred_row + '\n')
                                 